In [1]:
# from importib import reload
%load_ext autoreload

In [2]:
import os
import sys
sys.path.append('..')

from scripts.modeling.model_utils import split_and_save_datasets
from scripts.modeling.interpretability import explain_predictions
from scripts.modeling.trainer import train_ner_model, fine_tune_multiple_models

%autoreload 2

c:\Users\Darkles\10academy\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Darkles\10academy\env\Lib\site-packages\amseg-2.3-py3.12.egg\amseg\amharicSegmenter.py:55: SyntaxWarning: invalid escape sequence '\s'
c:\Users\Darkles\10academy\env\Lib\site-packages\amseg-2.3-py3.12.egg\amseg\amharicSegmenter.py:79: SyntaxWarning: invalid escape sequence '\s'
c:\Users\Darkles\10academy\env\Lib\site-packages\amseg-2.3-py3.12.egg\amseg\amharicSegmenter.py:55: SyntaxWarning: invalid escape sequence '\s'
c:\Users\Darkles\10academy\env\Lib\site-packages\amseg-2.3-py3.12.egg\amseg\amharicSegmenter.py:79: SyntaxWarning: invalid escape sequence '\s'
c:\Users\Darkles\10academy\env\Lib\site-packages\amseg-2.3-py3.12.egg\amseg\amharicNormalizer.py:87: SyntaxWarning: invalid escape sequence '\s'
c:\Users\Darkles\10academ

2025-01-23 14:10:17 - WARNING - From c:\Users\Darkles\10academy\env\Lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.

2025-01-23 14:10:17 - WARNING - From c:\Users\Darkles\10academy\env\Lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.

2025-01-23 14:10:17 - WARNING - From c:\Users\Darkles\10academy\env\Lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.



In [3]:
DATA_PATH = os.path.join('..', 'resources', 'data')
MODEL_PATH = os.path.join('..', 'resources', 'models')
labeled_dir = os.path.join(DATA_PATH, 'labeled')
splitted_dir = os.path.join(DATA_PATH, 'splitted')
model_output_dir = os.path.join(MODEL_PATH, "checkpoints", "ner_model")
os.makedirs(splitted_dir, exist_ok=True)
os.makedirs(model_output_dir, exist_ok=True)


dataset_path = ""
filepath = os.path.join(labeled_dir, 'labeled_messages.conll')


# Model and Data Constants
LEARNING_RATE = 5e-5
EPOCHS = 5
BATCH_SIZE = 8
MODEL_CHECKPOINT = "xlm-roberta-base"
LABEL_LIST = ["O", "B-Product", "I-Product", "B-LOC", "I-LOC", "B-PRICE", "I-PRICE"]


In [4]:
split_and_save_datasets(filepath, splitted_dir)

In [7]:
# Train without fine-tuning (full model training)
trainer_base = train_ner_model(
    data_path=splitted_dir,
    model_name=MODEL_CHECKPOINT,
    output_dir=os.path.join(model_output_dir, "full_model")
)

2025-01-23 14:11:53 - INFO - [INFO] Loading tokenized dataset from disk.
2025-01-23 14:11:53 - INFO - [INFO] Loading tokenized dataset from disk.
2025-01-23 14:11:53 - INFO - [INFO] Loading tokenized dataset from disk.
2025-01-23 14:11:53 - INFO - [INFO] Loading tokenized dataset from disk.
2025-01-23 14:11:53 - INFO - [INFO] Loading tokenized dataset from disk.
2025-01-23 14:11:53 - INFO - [INFO] Loading tokenized dataset from disk.
2025-01-23 14:11:53 - INFO - [INFO] Loading tokenized dataset from disk.
2025-01-23 14:11:53 - INFO - [INFO] Loading metadata from disk.
2025-01-23 14:11:53 - INFO - [INFO] Loading metadata from disk.
2025-01-23 14:11:53 - INFO - [INFO] Loading metadata from disk.
2025-01-23 14:11:53 - INFO - [INFO] Loading metadata from disk.
2025-01-23 14:11:53 - INFO - [INFO] Loading metadata from disk.
2025-01-23 14:11:53 - INFO - [INFO] Loading metadata from disk.
2025-01-23 14:11:53 - INFO - [INFO] Loading metadata from disk.
2025-01-23 14:11:53 - INFO - Loading data

In [ ]:
# Evaluate on the small evaluation set
eval_results = trainer_base.evaluate()
eval_results

Fine tuning

In [ ]:
# Fine-tune without LoRA (traditional fine-tuning)
trainer_no_lora = train_ner_model(
    data_path=dataset_path,
    model_name=MODEL_CHECKPOINT,
    output_dir=os.path.join(model_output_dir, "fine_tuned_without_lora"),
    use_peft=False,  # No PEFT
)

In [ ]:
# Evaluate on the small evaluation set
eval_results = trainer_no_lora.evaluate()
eval_results

In [ ]:
# Fine-tune with LoRA (parameter-efficient fine-tuning)
trainer_lora = train_ner_model(
    data_path=dataset_path,
    model_name=MODEL_CHECKPOINT,
    output_dir=os.path.join(model_output_dir, "fine_tuned_with_lora"),
    use_peft=True,  # Enable PEFT
)

In [ ]:
# Evaluate on the small evaluation set
eval_results = trainer_lora.evaluate()
eval_results

3 Fine tune multiple models

In [ ]:
# Define models and parameters
models = [
    "xlm-roberta-base",
    "Davlan/afroxlmr-large",
    "distilbert-base-multilingual-cased",
    "distilbert-base-uncased",
    "bert-base-multilingual-cased",
    "bert-base-uncased",
    "AfroXLMR",

]

# Fine-tune and compare models
params = {
    "learning_rate": 5e-5,
    "epochs": 5,
    "batch_size": 8,
}

# Fine-tune multiple models with LoRA
trainers, results_df = fine_tune_multiple_models(
    models=models,
    dataset_dir=splitted_dir,
    base_output_dir=model_output_dir,
    params=params,
    use_peft=True,
)
results_df

In [ ]:
trainers

# Interpretabiulity

In [ ]:

# Define the label mapping
label_map = {
    0: "O", 1: "B-PRICE", 2: "I-PRICE", 3: "B-LOC", 
    4: "I-LOC", 5: "B-PRODUCT", 6: "I-PRODUCT"
}

# Fine-tuned model and dataset path
model_name = "resources/models/comparison/xlm-roberta-base"
dataset_path = "resources/data/validation_data.txt"
example_idx = 5 

# Generate explanation
explain_predictions(model_name, dataset_path, example_idx, label_map)
